**INSTALLING THE REQ LIB**

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu evaluate rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 16.0 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import evaluate
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" # to check the device being used
device

'cuda'

**SELECTING AND DOWNLOADING THE PRE TRAINED MODEL**

In [ ]:
model_chkpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_chkpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_chkpt).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

**LOADING THE DATASET**

In [ ]:
dataset_samsum = load_dataset("knkarthick/samsum")
dataset_samsum

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14731 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

***PRE PROCESSING***

In [ ]:
def convertion_to_vectors(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)  # tokeninzing the dialouge
  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True) # tokeninzing the summary
  return(
      {
          'input_ids': input_encodings['input_ids'],
          'attention_mask': input_encodings['attention_mask'],
          'labels': target_encodings['input_ids']
      }
  )


In [ ]:
dataset_samsum_pt = dataset_samsum.map(convertion_to_vectors, batched=True) # applying map funtion to each item in the dataset

Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
dataset_samsum_pt["train"] # to view the splitting of dataset

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14731
})

In [ ]:
dataset_samsum_pt["train"]["id"][0]

'13818513'

In [ ]:
dataset_samsum_pt["train"]["dialogue"][0]

"Amanda: I baked  cookies. Do you want some?\nJerry: Sure!\nAmanda: I'll bring you tomorrow :-)"

In [ ]:
dataset_samsum_pt["train"]["summary"][0]

'Amanda baked cookies and will bring Jerry some tomorrow.'

In [ ]:
dataset_samsum_pt["train"]["input_ids"][0]

[12195,
 151,
 125,
 7091,
 3659,
 107,
 842,
 119,
 245,
 181,
 152,
 10508,
 151,
 7435,
 147,
 12195,
 151,
 125,
 131,
 267,
 650,
 119,
 3469,
 29344,
 1]

In [ ]:
dataset_samsum_pt["train"]["attention_mask"][0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
dataset_samsum_pt["train"]["labels"][0]

[12195, 7091, 3659, 111, 138, 650, 10508, 181, 3469, 107, 1]

**TRAINING THE MODEL**

In [ ]:
from transformers import DataCollatorForSeq2Seq # used to send data in batches if data is big and storage is not sufficient
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer # setting model arguments ir the rules on which the model will operate
training_args = TrainingArguments(
    output_dir="pegasus-samsun", # Directory to save the model checkpoints and logs
    num_train_epochs=1, # Number of training epochs
    warmup_steps=500, # Number of steps for the warmup phase
    per_device_train_batch_size=1, # Batch size for training on each device
    per_device_eval_batch_size=1, # Batch size for evaluation on each device
    weight_decay=0.01, # Amount of weight decay to apply
    logging_steps=10, # Log training progress every N steps
    eval_strategy="steps", # Evaluation strategy: "steps" to evaluate every `eval_steps`
    eval_steps=500, # Number of steps between evaluations
    save_steps=1e6, # Number of steps between saving checkpoints
    gradient_accumulation_steps=16, # Number of updates steps to accumulate before performing a backward/update pass
    report_to="none" # Disable W&B logging
)

In [ ]:
trainer = Trainer(model=model_pegasus,  # initializing the model
                  args=training_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"]
                  )

/tmp/ipython-input-2583004725.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus,


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,1.599500,1.483336


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4037: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=921, training_loss=1.832104067719591, metrics={'train_runtime': 2735.1092, 'train_samples_per_second': 5.386, 'train_steps_per_second': 0.337, 'total_flos': 5531715959930880.0, 'train_loss': 1.832104067719591, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.4253754615783691,
 'eval_runtime': 51.2863,
 'eval_samples_per_second': 15.95,
 'eval_steps_per_second': 15.95,
 'epoch': 1.0}

In [ ]:
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("/content/tokenizer")

**PREDICTION OF OUR MODEL**

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 128} # comman for setting the ouput rules
sample_txt= dataset_samsum["test"][0]["dialogue"] # taking the dialouge from the test data
refference= dataset_samsum["test"][0]["summary"]  # taking the actual summary according to the pegasus model
pipe=pipeline("summarization", model="pegasus-samsum-model", tokenizer=tokenizer) # creating the pipeline to tell the model wht to do

# result
print("Dialogue:")  # printing the actual dialouge
print(sample_txt)
print("\nReference Summary:") # printing the actual summary
print(refference)
print("\nModel Summary:") # printing the summary according to my model
print(pipe(sample_txt, **gen_kwargs)[0]["summary_text"])

Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda can't find Betty's number. Larry called Betty last time they were at the park together. Hannah wants Amanda to text Larry. Amanda will text Larry.
